# Importing some libraries

In [1]:
%matplotlib inline
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from PIL import Image

Most of the dlink camera will return the current frame if you try to download the link http://camera_IP/image.jpg.
So one easy way to make a time lapse is to download the image at the ling at regular interval.

Example in the cell below:

In [ ]:
import time
from IPython.html.widgets import FloatProgress
from urllib import request

interval=60 #seconds
imnum=100

f = FloatProgress(min=0, max=imnum-1)
display(f)
    
for i in range(imnum):
    
    dest_filename='ipcam_'+str(1000+i)[1:]+'.jpg'
    
    f = open(dest_filename, 'wb')
    f.write(request.urlopen("http://192.168.1.135/image.jpg").read())
    f.close()
    
    time.sleep(interval)
    f.description=str(i)
    f.value=i

The script above will save a sequence of jpg images in the folder running the notebook.

Another option is to upload the image as a document to a mongo db Gridfs collection.
In the next cell is an example of connecting to a local mongodb installation.

In [3]:
import pymongo
import gridfs
import mimetypes
import datetime
import time


# connect to the database & get a gridfs handle

conn=pymongo.MongoClient(127.0.0.1)
grid_fs = gridfs.GridFS(conn.test)
collection=conn.test.fs.files

def upload_image_to_db(orig_filename,dest_filename,grid_fs):
    ''' Upload the orig_filename image (assuming jpg) to the mongodb provided grid_fs
    and assign the name dest_filename. It also adds metadata: a dictionary containing
    the fields experiment_name,abstime and mmperpix. The function '''

    expname='timelapse_30_12_15'
    abstime=time.mktime(datetime.date.today().timetuple()) #usually I use unix time in h

    mime_type = 'image/jpeg'
    metadata = {'experiment_name': expname, 'abstime': abstime, 'mmperpix': 1};
    fileid = grid_fs.put(open(orig_filename,'rb'), contentType=mime_type, filename=dest_filename, info=info)
    return fileid

And with the upload_image_to_db the script could be

In [18]:
import time
from IPython.html.widgets import FloatProgress
from urllib import request

interval=60 #seconds
imnum=100

f = FloatProgress(min=0, max=imnum-1)
display(f)
    
for i in range(imnum):
    
    f = open('timelapse_im', 'wb')
    f.write(request.urlopen("http://192.168.1.135/image.jpg").read())
    f.close()

    dest_filename='ipcam_'+str(1000+i)[1:]+'.jpg'

    upload_image_to_db('timelapse_im',dest_filename)
    time.sleep(interval)
    f.description=str(i)
    f.value=i

I have some good way to then explore the database I'll post a different notebook